In [776]:
%run ../config/config.py
#from config.config import *
import pandas as pd
import numpy as np
import debug
import cv2

In [777]:
image = cv2.imread(BLACK_YELLOW)
image_size = (SIZE,SIZE)
image = cv2.resize(image,image_size)

In [778]:
def _detect_edges(image):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, THRESHOLD1, THRESHOLD2, EDGES)
    return edges

edges_image = _detect_edges(image)

In [779]:
def _generate_edges_lines(canny):
    
    lines = cv2.HoughLines(canny, 
                            rho = RHO_RESOLUTION, 
                            theta =  THETA_RESOLUTION, 
                            threshold = HOUGH_THRESHOLD)
    
    #Format dataset
    lines = lines.reshape(len(lines),2)
    
    coor_lines = []
    for line in lines:
        rho,theta = line
        a = np.cos(theta)
        b = np.sin(theta)
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 +DISTANCE*(-b))
        y1 = int(y0 +DISTANCE*(a))
        x2 = int(x0 -DISTANCE*(-b))
        y2 = int(y0 -DISTANCE*(a))   
        coor_lines.append(((x1,y1),(x2,y2)))
    
    return np.array(coor_lines)
    
lines = _generate_edges_lines(edges_image)

In [780]:
from sklearn.cluster import KMeans

def _get_lines_means(points, direction, n_clusters = 9):
    
    means = []

    for lines in points:
    
        kmeans = KMeans(n_clusters = n_clusters, n_init=10)
        kmeans.fit(lines)
        labels = kmeans.labels_
    
        unique_labels = list(range(n_clusters))
    
        cluster_means = []
    
        for label in unique_labels: 
            cluster_data = lines[labels == label] 
            cluster_mean = np.round(sum(cluster_data)/len(cluster_data))  
            cluster_means.append(cluster_mean)
        
        cluster_means = np.array(cluster_means)
        sorted_index = np.argsort(cluster_means[:, direction])
        
        means.append(cluster_means[sorted_index])
 
    return means


In [781]:


def _get_vertical_averages(lines):
    
    vertical_lines = lines[lines[:,0,1] > SIZE]
    
    initial_points = vertical_lines[:,1][vertical_lines[:,1,1] < -SIZE]
    final_points = vertical_lines[:,0][vertical_lines[:,0,1] > SIZE] 
    
    averages = _get_lines_means((initial_points, final_points), VERTICAL)
    
    return np.stack(averages, axis=1)

def _get_horizontal_averages(lines):

    horizontal_lines = lines[lines[:,1,0] > SIZE]
    
    initial_points = horizontal_lines[:,0][horizontal_lines[:,0,0] < -SIZE]
    final_points = horizontal_lines[:,1][horizontal_lines[:,1,0] > SIZE]
    
    averages = _get_lines_means((initial_points, final_points), HORIZONTAL)
    
    return np.stack(averages, axis=1)
    
vertical_lines = _get_vertical_averages(lines).astype(int)
horizontal_lines = _get_horizontal_averages(lines).astype(int)

lines = np.concatenate((vertical_lines,horizontal_lines), axis=0)

#debug.lines(image,lines, size = 3);


In [782]:
def _detect_inner_corners(lines, corners = MAX_CORNERS):
    
    black_image = np.zeros((SIZE, SIZE, EDGES), dtype=np.uint8)
    debug.lines(black_image,lines, size = 1);
    black_image = _detect_edges(black_image)
    points = cv2.goodFeaturesToTrack(black_image, corners, QUALITY, MIN_DISTANCE)
    return points.reshape(len(points),2)
#debug.lines(image,lines, size = 1);
points =  _detect_inner_corners(lines)


In [783]:
"""import itertools

def line_intersections(line1, line2):
    x1, y1 = line1[0]
    x2, y2 = line1[1]
    x3, y3 = line2[0]
    x4, y4 = line2[1]

    # Calculate the denominator for the intersection point calculation
    denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)
    if  0 in denominator:
        return None  # Lines are parallel and do not intersect

    # Calculate the intersection point
    px = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denominator
    py = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denominator

    return (px, py)

def line_intersection(lines):

    lines = list(itertools.combinations(lines, 2))
    intersections = []

    for i in range(len(lines)):
        intersection = line_intersections(lines[0], lines[1])
        if intersection is not None:
                intersections.append(intersection)
            
    return np.array(points)



#print(lines)
points = line_intersection(lines)

print(points)

lower_limit = -1000
upper_limit = 1000
#points = points.astype(int)
# Create a mask to filter points within the range
#mask = (points[:, 0] >= lower_limit) & (points[:, 0] <= upper_limit) & (points[:, 1] >= lower_limit) & (points[:, 1] <= upper_limit)

#points = points[mask]

print(len(points))
debug.points(image,points, color = (0,255,0), size = 7);

#print(len(sorted_intersections))"""

'import itertools\n\ndef line_intersections(line1, line2):\n    x1, y1 = line1[0]\n    x2, y2 = line1[1]\n    x3, y3 = line2[0]\n    x4, y4 = line2[1]\n\n    # Calculate the denominator for the intersection point calculation\n    denominator = (x1 - x2) * (y3 - y4) - (y1 - y2) * (x3 - x4)\n    if  0 in denominator:\n        return None  # Lines are parallel and do not intersect\n\n    # Calculate the intersection point\n    px = ((x1 * y2 - y1 * x2) * (x3 - x4) - (x1 - x2) * (x3 * y4 - y3 * x4)) / denominator\n    py = ((x1 * y2 - y1 * x2) * (y3 - y4) - (y1 - y2) * (x3 * y4 - y3 * x4)) / denominator\n\n    return (px, py)\n\ndef line_intersection(lines):\n\n    lines = list(itertools.combinations(lines, 2))\n    intersections = []\n\n    for i in range(len(lines)):\n        intersection = line_intersections(lines[0], lines[1])\n        if intersection is not None:\n                intersections.append(intersection)\n            \n    return np.array(points)\n\n\n\n#print(lines)\npoints

In [784]:
def _get_middle_squares(lines):

    middle_lines = []

    for i in range(17):
        
        if i == 8: continue
        
        mean = (lines[i] +lines[i+1])/2
        
        middle_line = mean.astype(int).tolist()
        middle_lines.append(middle_line)
        
    corners = _detect_inner_corners(middle_lines, 64)
    return corners

middle_points = _get_middle_squares(lines)


In [785]:
def _distance(a, b):
    
    x_diff = abs(a[0] - b[0])
    y_diff = abs(a[1] - b[1])
    
    distance = np.sqrt(x_diff**2 + y_diff**2)
    
    return distance

In [786]:
def _get_grid(points, lines):
    grid_points = _detect_inner_corners(lines)
    
    grid = []
    
    for middle_point in points:
        
        distances = []
        
        for point in grid_points:
            
            distances.append(_distance(middle_point, point))
            
        index = np.argsort(distances)
        
        order = grid_points[index]
        
        final = order[:4].tolist()
        
        grid.append(final)
        
    #grid_index = np.argsort(middle_point)
    return grid
    #print(_distance(points, grid_points))


grid = _get_grid(middle_points, lines)
debug.points(image, grid[24], color = RED, size = 8)
print(grid)

[[[220.0, 263.0], [219.0, 345.0], [297.0, 264.0], [297.0, 346.0]], [[69.0, 122.0], [63.0, 193.0], [147.0, 124.0], [145.0, 194.0]], [[138.0, 579.0], [137.0, 645.0], [215.0, 645.0], [216.0, 578.0]], [[64.0, 580.0], [137.0, 645.0], [63.0, 646.0], [138.0, 579.0]], [[294.0, 578.0], [294.0, 644.0], [370.0, 644.0], [370.0, 577.0]], [[216.0, 578.0], [215.0, 645.0], [294.0, 644.0], [294.0, 578.0]], [[517.0, 643.0], [517.0, 576.0], [444.0, 577.0], [438.0, 643.0]], [[438.0, 643.0], [370.0, 577.0], [444.0, 577.0], [370.0, 644.0]], [[659.0, 642.0], [588.0, 576.0], [660.0, 575.0], [588.0, 643.0]], [[588.0, 576.0], [588.0, 643.0], [517.0, 576.0], [517.0, 643.0]], [[138.0, 579.0], [134.0, 509.0], [216.0, 578.0], [216.0, 509.0]], [[134.0, 509.0], [138.0, 579.0], [64.0, 580.0], [64.0, 508.0]], [[517.0, 576.0], [518.0, 510.0], [445.0, 510.0], [444.0, 577.0]], [[444.0, 577.0], [371.0, 510.0], [445.0, 510.0], [370.0, 577.0]], [[370.0, 577.0], [371.0, 510.0], [295.0, 509.0], [294.0, 578.0]], [[216.0, 509.0]

In [787]:

cv2.imshow("Image", image)
cv2.waitKey(0)
cv2.destroyAllWindows()